### Observación de las métricas de almacenamiento y acceso de S3 mediante Storage Lens

Todas las cuentas de AWS tienen un panel de control predeterminado asociado que muestra las métricas gratuitas disponibles a través de S3 Storage Lens. La activación de las métricas avanzadas le proporciona una visión más profunda de su uso de S3 y también le ofrece recomendaciones de ahorro de costes que puede llevar a cabo para optimizar los costes. Puede utilizar el panel de control predeterminado y/o crear el suyo propio. El resto de estos pasos le mostrarán cómo crear el suyo propio.

In [6]:
import boto3
import json
import random
from pprint import pprint 

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
s3control = boto3.client('s3control', region_name=region_aws)

iam = boto3.resource('iam')
iam_client = boto3.client('iam')

In [2]:
# Create a bucket
bucket_name = 'cookbook-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket_name)
print("Bucket name: {}".format(bucket_name))

# Por defecto cuando creamos un bucket, las opciones de **Block all public access** (Bloquear todo el acceso público) 
# están desactivadas por defecto.

response = s3_client.put_public_access_block(
        Bucket=bucket_name,
        PublicAccessBlockConfiguration={
            'BlockPublicAcls': True,
            'IgnorePublicAcls': True,
            'BlockPublicPolicy': True,
            'RestrictPublicBuckets': True
        }
)

Bucket name: cookbook-895493


In [7]:
# Get account ID and bucket ARN
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_arn = 'arn:aws:s3:::{}'.format(bucket_name)

In [19]:
policy_s3_lens = {
     "Id": "SampleS3StorageLensConfiguration", # Use this property to identify S3 Storage Lens configuration.
     "AccountLevel": {
        "ActivityMetrics": {
           "IsEnabled": True
        },
        "BucketLevel": {
           "ActivityMetrics": {
              "IsEnabled":True #Mark this as false if you only want Free Metrics metrics.
           },
           "PrefixLevel":{
              "StorageMetrics":{
                 "IsEnabled":True, #Mark this as false if you only want Free Metrics metrics.
                 "SelectionCriteria":{
                    "MaxDepth":5,
                    "MinStorageBytesPercentage":1.25,
                    "Delimiter":"/"
                 }
              }
           }
        }
     },
     "Exclude": { #Replace with include if you prefer to include regions.
        "Regions": [
           "eu-west-1"
        ],
        "Buckets": [ #This attribute is not supported for  Organizations-level configurations.
           "arn:aws:s3:::source_bucket1" 
        ]
     },
     "IsEnabled": True, #Whether the configuration is enabled
     "DataExport": { #Details about the metrics export
        "S3BucketDestination": {
           "OutputSchemaVersion": "V_1",
           "Format": "CSV", #You can add "Parquet" if you prefer.
           "AccountId": account_id,
           "Arn": bucket_arn, # The destination bucket for your metrics export must be in the same Region as your S3 Storage Lens configuration. 
           "Prefix": "/data",
           "Encryption": {
              "SSES3": {}
           }
        }
     }
  }

Mediante el parámetro `DataExport` puedes habilitar una exportación automatizada para exportar periódicamente tus métricas a formatos CSV y Apache Parquet y enviarlas a un bucket S3 de tu elección para ejecutar tus propios informes y visualizaciones.

In [23]:
# Enable S3 Storage Lens
response = s3control.put_storage_lens_configuration(
    AccountId=account_id,
    ConfigId='SampleS3StorageLensConfiguration',
    StorageLensConfiguration=policy_s3_lens,
    Tags=[{
        "Key": "key1",
        "Value": "value1"
    },
    {
        "Key": "key2",
        "Value": "value2"
    }]
)

pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Mon, 17 Oct 2022 12:14:21 GMT',
                                      'server': 'AmazonS3',
                                      'x-amz-id-2': '0DKCHkdGvs9pmWp/pCZvm92Gbv3SrMRj5brYfuXtulT6rB5YL1DQeqee12Ozhm6FtlWJtZz9f9A=',
                                      'x-amz-request-id': 'V4GB9H55CNKSA822'},
                      'HTTPStatusCode': 204,
                      'HostId': '0DKCHkdGvs9pmWp/pCZvm92Gbv3SrMRj5brYfuXtulT6rB5YL1DQeqee12Ozhm6FtlWJtZz9f9A=',
                      'RequestId': 'V4GB9H55CNKSA822',
                      'RetryAttempts': 0}}


Las métricas avanzadas pueden tardar hasta 48 horas en empezar a acumularse para tus patrones de uso y acceso. Mientras tanto, puedes ver el panel por defecto para las métricas gratuitas asociadas a tu uso de S3 para todos los buckets de tu cuenta.


In [24]:
# Get S3 Storage Lens
response = s3control.get_storage_lens_configuration(
    AccountId=account_id,
    ConfigId='SampleS3StorageLensConfiguration'
)

pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '1142',
                                      'date': 'Mon, 17 Oct 2022 12:21:10 GMT',
                                      'server': 'AmazonS3',
                                      'x-amz-id-2': 'GaPy7B5NwdHF9G/pNCgKhBB6NaQRl68X+MxjFDsULvfYUh95j0P6Je19eqVDSCkZ8tbdXGqn5aA=',
                                      'x-amz-request-id': 'FBM551Q3NNE8XC17'},
                      'HTTPStatusCode': 200,
                      'HostId': 'GaPy7B5NwdHF9G/pNCgKhBB6NaQRl68X+MxjFDsULvfYUh95j0P6Je19eqVDSCkZ8tbdXGqn5aA=',
                      'RequestId': 'FBM551Q3NNE8XC17',
                      'RetryAttempts': 0},
 'StorageLensConfiguration': {'AccountLevel': {'ActivityMetrics': {'IsEnabled': True},
                                               'BucketLevel': {'ActivityMetrics': {'IsEnabled': True},
                                                               'PrefixLevel': {'StorageMetrics': {'IsEnabled': True,
                        

Puede profundizar en las métricas de "Eficiencia de costes" y "Protección de datos" desde el panel de control. Después de algún tiempo, podrá ver datos históricos que le permitirán tomar medidas para mover sus objetos a niveles de almacenamiento que satisfagan sus necesidades de patrones de acceso a los datos y requisitos de disponibilidad.

#### Discusión
S3 fue uno de los primeros servicios de AWS, y como resultado, muchos clientes han estado utilizando S3 durante mucho tiempo. A medida que el uso de almacenamiento de los clientes creció exponencialmente, la capacidad de analizar lo que se está almacenando se convirtió en una capacidad clara y deseada. S3 Storage Lens le ofrece la posibilidad de "ver" el uso de S3 en sus cuentas de AWS. Analizar el uso de los cubos, observar los costes de almacenamiento y descubrir anomalías (por ejemplo, fragmentos de carga multiparte no eliminados) son sólo algunos de los muchos casos de uso que ofrece S3 Storage Lens.

Con Storage Lens, puede descubrir dónde se almacenan sus objetos con un panel visual respaldado por un potente motor de análisis, de modo que pueda realizar ajustes para optimizar los costes sin afectar al rendimiento. También puede habilitar métricas avanzadas en su tablero para obtener información más profunda y recomendaciones de ahorro de costes para sus buckets de S3.

**NOTA:** S3 Storage Lens utiliza métricas para ayudarle a visualizar su uso y actividad. Hay métricas gratuitas disponibles y métricas avanzadas que también le dan recomendaciones sobre su uso. Para más información sobre los diferentes tipos de métricas y sus costes asociados, consulte el [documento de soporte](https://oreil.ly/HQcLH).